### Shanghai Rental Data 

This data is scrapied from Lianjia.com (as of 10/01/2020). The csv files contains name, rental type, # of beds, price, longitude, latitude, balcony, deposit, apartment, rent_description, touring, transportation, independent bath, furnished, new listing, size, orientation, leve, elevator, parking and amenities information.

Attributes:
- name: name of listing
- type: sublease or entire leasing (all entire)
- beds: bedroom number
- price
- lon/lat: coordinates
- balcony, deposit(whether has deposit policy), apartment, rent_description, touring availability, close to transportation, independent bath, furnished, new listing: NO-0, YES-1.
- size: square meters
- orientation: facing of the windows, south-1, southeast-2, east-3, north-4, southwest-5, west-6, northwest-7, northeast-8, unknown-0
- level: levels of the listing, basement-0, low level(1-15)-1, mid level(15-25)-2, high level(>25)-3
- parking: no parking-0, extra cost avalaible-1, free parking-2
- amenities: amenities number 

In [124]:
import pandas as pd
import numpy as np

In [131]:
df = pd.read_csv('lianjia_shanghai.csv', sep =',', encoding='utf_8_sig', header=None)

In [132]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,三兴小区,整租,1,4500.0,121.466522,31.258257,0,0,0,0,...,1,0,1,0,31,北,低楼层/7层,无,暂无数据,5
1,华龙大酒店,整租,1,3700.0,121.458494,31.262110,0,0,0,0,...,1,0,0,0,43,南,中楼层/16层,无,暂无数据,0
2,协和大厦,整租,2,8500.0,121.464014,31.265904,0,0,0,0,...,1,0,0,0,92,南,中楼层/20层,有,暂无数据,7
3,安源小区,整租,2,8200.0,121.457048,31.251189,0,0,0,0,...,0,0,1,0,78,南,低楼层/21层,有,暂无数据,8
4,和泰花园,整租,2,9000.0,121.457915,31.263993,0,0,0,0,...,1,0,0,0,93,南,低楼层/17层,有,租用车位,8


### ETL

ETL process, clean and translate the dataframe into English.

In [138]:
def cleanRental(df):
    # drop duplicates
    df_copy = df.copy()
    df_copy = df_copy.drop_duplicates()
    #rename columns 
    cols = ['name', 'type', 'beds', 'price', 'lon', 'lat', 'balcony', 'deposit', 'apartment', 'rent_description', 
            'touring', 'close_to_transportation', 'independent_bath', 'furnished', 'new_listing', 'size', 
            'orientation', 'level', 'elevator', 'parking', 'amenities']
    df_copy.columns = cols
    # remove type since it's all entire apartment
    df_copy.drop('type', axis=1, inplace=True)
    # clean beds type unreadable
    df_copy.drop(10811, axis=0, inplace=True)
    # translate orientation
    df_copy.orientation = df_copy.orientation.apply(lambda x: x.split(' ')[0])
    rep = {'南': 1, '东南': 2, '东': 3, '北': 4, '西南': 5, '西': 6, '西北': 7, '东北':8, '未知': 0}
    df_copy.orientation.replace(rep, inplace=True)
    # clean levels 
    df_copy.level = df_copy.level.apply(lambda x: x.split('/')[0])
    df_copy.level.replace({'地下室': 0, '低楼层': 1, '中楼层': 2, '高楼层':3}, inplace=True)
    #df_copytranslate elevator
    df_copy.elevator.replace({'无':0, '有':1}, inplace=True)
    # translate parking info
    df_copy.parking.replace({'暂无数据': 0, '租用车位': 1, '免费使用': 2}, inplace=True)
    return df_copy

In [139]:
df_clean = cleanRental(df)

In [140]:
df_clean.head()

,name,beds,price,lon,lat,balcony,deposit,apartment,rent_description,touring,close_to_transportation,independent_bath,furnished,new_listing,size,orientation,level,elevator,parking,amenities
0,三兴小区,1,4500.0,121.466522,31.258257,0,0,0,0,1,1,0,1,0,31,4,1,0,0,5
1,华龙大酒店,1,3700.0,121.458494,31.262110,0,0,0,0,1,1,0,0,0,43,1,2,0,0,0
2,协和大厦,2,8500.0,121.464014,31.265904,0,0,0,0,1,1,0,0,0,92,1,2,1,0,7
3,安源小区,2,8200.0,121.457048,31.251189,0,0,0,0,0,0,0,1,0,78,1,1,1,0,8
4,和泰花园,2,9000.0,121.457915,31.263993,0,0,0,0,0,1,0,0,0,93,1,1,1,1,8


In [141]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10994 entries, 0 to 10998
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     10994 non-null  object 
 1   beds                     10994 non-null  object 
 2   price                    10994 non-null  float64
 3   lon                      10994 non-null  float64
 4   lat                      10994 non-null  float64
 5   balcony                  10994 non-null  int64  
 6   deposit                  10994 non-null  int64  
 7   apartment                10994 non-null  int64  
 8   rent_description         10994 non-null  int64  
 9   touring                  10994 non-null  int64  
 10  close_to_transportation  10994 non-null  int64  
 11  independent_bath         10994 non-null  int64  
 12  furnished                10994 non-null  int64  
 13  new_listing              10994 non-null  int64  
 14  size                  

In [144]:
df_clean.beds.value_counts()

2    4227
3    2743
1    2732
4     870
5     312
6      59
7      28
8      12
0       7
9       4
Name: beds, dtype: int64

### ETL

In [143]:
# clean beds 
df.beds.value_counts()

AttributeError: 'DataFrame' object has no attribute 'beds'

In [86]:
# drop iregular bed
df.drop(10813, axis=0, inplace=True)
df.beds.value_counts()

2    4228
3    2744
1    2734
4     870
5     312
6      59
7      28
8      12
0       7
9       4
Name: beds, dtype: int64

In [87]:
# check price
df.price.describe()

count      10998
unique       376
top       4500.0
freq         424
Name: price, dtype: object

In [88]:
# clean orientation
df.orientation = df.orientation.apply(lambda x: x.split(' ')[0])

In [89]:
rep = {'南':1, '东南': 2, '东':3, '北': 4, '西南':5, '西': 6, '西北': 7, '东北':8, '未知':0}
df.orientation.replace(rep, inplace=True)

In [90]:
df.orientation.value_counts()

1    9968
2     290
3     242
4     211
5     130
6      96
7      27
8      23
0      11
Name: orientation, dtype: int64

In [91]:
# check level
df.level = df.level.apply(lambda x: x.split('/')[0])

In [92]:
df.level.value_counts()

高楼层    3724
低楼层    3656
中楼层    3616
地下室       2
Name: level, dtype: int64

In [93]:
df.level.replace({'地下室': 0, '低楼层': 1, '中楼层': 2, '高楼层':3}, inplace=True)
df.level.value_counts()

3    3724
1    3656
2    3616
0       2
Name: level, dtype: int64

In [94]:
# check elevator
df.elavator.value_counts()
df.elavator.replace({'无':0, '有':1}, inplace=True)

In [95]:
df.elavator.value_counts()

0    5520
1    5478
Name: elavator, dtype: int64

In [96]:
# check parking
df.parking.value_counts()

暂无数据    7706
租用车位    2679
免费使用     613
Name: parking, dtype: int64

In [97]:
df.parking.replace({'暂无数据': 0, '租用车位':1, '免费使用': 2}, inplace=True)
df.parking.value_counts()

0    7706
1    2679
2     613
Name: parking, dtype: int64